In [1]:
import arrow
import argparse            # Construct the argument parser
import logging
import numpy as np
import pandas as pd
import pyodbc
import time
import warnings

warnings.filterwarnings('ignore')

from datetime import datetime
from humanfriendly import format_timespan
from sqlalchemy import create_engine
from tqdm import tqdm

In [2]:
from osisoft.pidevclub.piwebapi.pi_web_api_client import PIWebApiClient
from osisoft.pidevclub.piwebapi.models import PIStreamValues, PITimedValue

from PiHelper import *

In [3]:
server = '192.168.5.191'
database = 'Runtime'
username = 'pi'
password = '1m4dm1n'
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [23]:
piwebapi_url = 'https://192.168.5.190/piwebapi'
client = PIWebApiClient(piwebapi_url, useKerberos=False, username="administrator", password="Spc12345", verifySsl=False) 
piHelper = PiHelper(client)

In [5]:
# Read model configuration file
df_conf = pd.read_csv('agincourt_resources.csv')

# Filter model configuration based on object type
#object_type = "ObjectType=='Attribute'"
#df_conf = df_conf.query(object_type)
df_conf = df_conf.reset_index(drop=True)


In [ ]:
df_conf

In [6]:
# Create start date and end date table for datestamp selection
start = arrow.get("20220101 00:00:00")
end = arrow.get("20220315 23:59:59")
start_list = []
end_list = []

for r in arrow.Arrow.span_range('days', start, end):
    sdate = r[0].floor('day').format('YYYYMMDD HH:mm:ss')
    start_list.append(sdate)
    edate = r[1].floor('second').shift(days=+6).format('YYYYMMDD HH:mm:ss')
    end_list.append(edate)

df_date = pd.DataFrame(list(zip(start_list, end_list)), columns =['start_date', 'end_date'])
#df_date['start_date'] = pd.to_datetime(df_date['start_date'])
#df_date['end_date'] = pd.to_datetime(df_date['end_date'])

l = list(np.arange(0,len(df_date),7))
df_date = df_date.iloc[l].sort_values('start_date',ascending=False).reset_index()
df_date = df_date.drop(columns=['index'])

In [ ]:
df_date

In [7]:
# For test
tagname = str(df_conf['Name'][0])            # tag_path = '\\\\PISERVER\\Database1\\PCS0210PN001_Crushing|MAR.M0210_FE001_MI_sPV'
af_path = str(df_conf['AFPath'][0])

#logging.basicConfig(filename=tagname+'.log', filemode='a', level=logging.INFO, format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S')
tagname, af_path

('MAR.M0280_Dose_rateSP_sPV',
 '\\\\WS-AUTO\\PTAR\\Agincourt Resources\\Flocculant_plant|MAR.M0280_Dose_rateSP_sPV')

In [8]:
startdate = df_date['start_date'][len(df_date)-1] #"20220330 00:00:00"
enddate = df_date['end_date'][len(df_date)-1] # "20220315 23:59:59"
startdate, enddate

('20220101 00:00:00', '20220107 23:59:59')

In [9]:
%%time
# Query to WW
sql = "SELECT DateTime, Value \
 FROM History \
 WHERE History.TagName = '{}' \
 AND wwRetrievalMode = 'Delta' \
 AND wwResolution = 1000 \
 AND wwQualityRule = 'Extended' \
 AND Quality = 0 \
 AND wwVersion = 'Latest' \
 AND DateTime >= '{}' \
 AND DateTime <= '{}' ".format(tagname,startdate,enddate)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 7.63 µs


In [10]:
%%time
t0 = time.time()
# Get data from WW Historian
df = pd.read_sql(sql, cnxn)
t1 = time.time()
execution_time = format_timespan(t1 - t0, True, max_units=2)
print("Exection time = {}".format(execution_time))

Exection time = 13 milliseconds, 646 microseconds and 125.79 nanoseconds
CPU times: user 2.72 ms, sys: 218 µs, total: 2.94 ms
Wall time: 14 ms


In [30]:
df

,DateTime,Value
0,2022-01-04 11:12:52,125.0
1,2022-01-05 16:14:03,130.0
2,2022-01-06 13:27:35,135.0


In [11]:
%%time
# Get values
values = df['Value']

# Get timestamp
timestamps = df['DateTime']

CPU times: user 29 µs, sys: 0 ns, total: 29 µs
Wall time: 31.5 µs


In [27]:
%%time
# Write data to PI tag
response = piHelper.insertTimeSeriesValues(af_path, values, timestamps)

CPU times: user 21.2 ms, sys: 0 ns, total: 21.2 ms
Wall time: 48.1 ms


In [28]:
print(str(response))

({'items': None, 'links': None}, 202, {'Transfer-Encoding': 'chunked', 'Server': 'Microsoft-HTTPAPI/2.0', 'X-Frame-Options': 'SAMEORIGIN', 'Content-Security-Policy': "default-src 'self'; object-src 'none'", 'X-Content-Type-Options': 'nosniff', 'content-type': 'application/json', 'content-encoding': 'gzip', 'Date': 'Fri, 08 Apr 2022 22:16:42 GMT'})


In [29]:
response

({'items': None, 'links': None},
 202,
 {'Transfer-Encoding': 'chunked', 'Server': 'Microsoft-HTTPAPI/2.0', 'X-Frame-Options': 'SAMEORIGIN', 'Content-Security-Policy': "default-src 'self'; object-src 'none'", 'X-Content-Type-Options': 'nosniff', 'content-type': 'application/json', 'content-encoding': 'gzip', 'Date': 'Fri, 08 Apr 2022 22:16:42 GMT'})

In [ ]:
if 202 in response:
    print("OK")

In [ ]:
# Read data
#tag_path = "pi:\\PISERVER\\" + tagname
attr_pi = "af:" + af_path

end = arrow.get(enddate,'YYYYMMDD HH:mm:ss').format('YYYY-MM-DD HH:mm:ss')
start = arrow.get(startdate,'YYYYMMDD HH:mm:ss').format('YYYY-MM-DD HH:mm:ss')

# Get data from PI Historian based on source_tag
#print("Get data from {}, {} ...".format(name, source_tag))
dp = piHelper.getInterpolatedData(path=attr_pi, start_time=start, end_time=end, interval='1M')

In [ ]:
dp.head(10)